In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("schedule.csv")

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,15,16,17,19,30,31,32,36,44,45,47,48,49,50,51,52,53,55,56,59,64,72,73,74,75,77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cropped = data.sample(2000)

In [6]:
cropped.head()

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,entity_type,entity_type_desc,...,schedule_type,sub_id,pdf_url,line_number_label,payee_prefix,payee_suffix,payee_employer,payee_occupation,ref_disp_excess_flg,comm_dt
413257,C00193433,EMILY'S LIST,2019,M2,201902149145514806,21B,SB21B-326578,1315964,IND,INDIVIDUAL,...,SB,4022720191644221462,http://docquery.fec.gov/cgi-bin/fecimg/?201902...,Other Federal Operating Expenditures,NaN,NaN,NaN,NaN,NaN,NaN
417290,C00401224,ACTBLUE,2019,MY,201907299159563504,23,SB23_145192899,1344765,PAC,POLITICAL ACTION COMMITTEE,...,SB,4082820191116017111,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions to Federal Candidates/Committees...,NaN,NaN,NaN,NaN,NaN,NaN
51714,C00401224,ACTBLUE,2019,MY,201907299159552920,23,SB23_144417961,1344765,PAC,POLITICAL ACTION COMMITTEE,...,SB,4082820191116161623,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions to Federal Candidates/Committees...,NaN,NaN,NaN,NaN,NaN,NaN
423258,C00401224,ACTBLUE,2019,MY,201907299160110486,23,SB23_145146146,1344765,PAC,POLITICAL ACTION COMMITTEE,...,SB,4082820191119273326,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions to Federal Candidates/Committees...,NaN,NaN,NaN,NaN,NaN,NaN
145924,C00401224,ACTBLUE,2019,MY,201907299160807901,23,SB23_144322262,1344765,PAC,POLITICAL ACTION COMMITTEE,...,SB,4082820191114732311,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions to Federal Candidates/Committees...,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
cropped.to_csv("cropped_data.csv")

In [8]:
categories = data['disbursement_description'].unique()

In [9]:
dataset = pd.DataFrame({'Categories': categories})

In [10]:
dataset.head(20)

,Categories
0,DIGITAL CONSULTING/FUNDRAISING
1,ACCOUNT ANALYSIS CHASE
2,VOIDED ORIGINAL DATE: 10/12/18
3,VOIDED ORIGINAL DATE: 04/13/18; LATER PAID WIT...
4,VOIDED ORIGINAL DATE: 03/13/18; PAID ONLINE 3/...
5,VOIDED ORIGINAL DATE: 04/30/18; VENDOR LOST CH...
6,"VOIDED CONTRIBUTION, ORIGINALLY DISCLOSED 10/2..."
7,PRINTING
8,GENERAL CONSULTING
9,COMPLIANCE SERVICES


In [11]:
dataset.to_csv("categories.csv")

In [12]:
dropped = dataset.dropna()
dataset = dropped

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [14]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [15]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/ilica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ilica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [19]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in dataset["Categories"]:
    print(i)
    try:
        allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
        totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
        
        allwords_tokenized = tokenize_only(i)
        totalvocab_tokenized.extend(allwords_tokenized)
        
    except:
        print("skipping")

DIGITAL CONSULTING/FUNDRAISING
ACCOUNT ANALYSIS CHASE
VOIDED ORIGINAL DATE: 10/12/18
VOIDED ORIGINAL DATE: 04/13/18; LATER PAID WITH CR
VOIDED ORIGINAL DATE: 03/13/18; PAID ONLINE 3/19/2
VOIDED ORIGINAL DATE: 04/30/18; VENDOR LOST CHECK,
VOIDED CONTRIBUTION, ORIGINALLY DISCLOSED 10/29/18
PRINTING
GENERAL CONSULTING
COMPLIANCE SERVICES
CREDIT CARD PROCESSING FEES
MERCHANT FEES
RECONCILING CURRENT ELECTION CYCLE
RESETTING CARRYOVER EXPENSES TO THIS ELECTION CYCLE
RESETTING OPERATING EXPENSES FOR CURRENT ELECTION CYCLE
RESETTING EXPENSES FOR NEW CAMPAIGN PUT IN WRONG POLITICAL YEAR
NO TRANSACTIONS IN JANUARY
ADVERTISEMENT AND POLITICAL CONTRIBUTIONS
UBI CASE STUDY
IN-KIND: UBI CASE STUDY
REIMBURSEMENT (VENDORS THAT AGGREGATE OVER $200 LISTED BELOW)
TRANSPORTATION
MEALS
SOFTWARE
RENT
WEBSITE
DATABASE
ADJUST BOOK BALANCE OF CASH TO BANKVOIDED CHECKS UNABLE TO TRACE ORIG DATE
TO CORRECT BEGINNING CASH
PCGPAC CONTRIBUTION TO FILEMON VELA FOR CONGRESS
CREDIT PROCESSING FEES
CONTRIBUTION IN-KIN

SOCIAL MEDIA CONSULTING
FUNDRAISING
PAYROLL - TAXES
PAYROLL - INVOICE
UNLIMITED TRANSFER
SIGN PICK-UP TRUCK LEASE
OVERHEAD; CAMPAIGN DONATION WEBSITE
CELL PHONE SERVICES
SEE MEMO ITEM BELOW
FUNDRAISING FEE
BOOKKEEPING
SOCIAL MEDIA
TRAVEL EXPENSE - TRANSPORTATION
MILEAGE
MONTHLY STIPEND - OWED TO COMMITTEE
FUNDRAISING DIRECT MAIL PRINTING
DONATION PROCESSING, CAGING, BOOKKEEPING
PROFIT SHARE OF 2018 FUNDRAISER
WIRELESS PHONE/DATA SERVICE
DAILY NEWSPAPERS - MONTHLY PAYMENT
LEGAL ADVICE, SERVICES
FINANCIAL SERVICES
CONDUIT FEE - EARMAKED CONTRIBUTION
FUNDRAISING TRANSPORTATION
RETREAT
PHONES AND CABLE
MERCHANT FEE
2019 NATIONAL COMMITTEE SUPPORT
2019 NRCC SUPPORT
EVENT:  FEBRUARY 14-16, 2019
BANK CHECKS
CONTRACT - CLERICAL
ACCOUNTIG SERVICES
CAMPAIGN STRATEGY
CREDIT CARD PROCESSING
CAGING AND ESCROW
COMPUTER AND INTERNET EXPENSES
CREDIT CARD PMT PROCESSING AND VERIFICATIONS
SOFTWARE LICENSING
SECURITY/GET OUT THE VOTE RALLY
TELEPHONE EXPENSE
PAYROLL FEE
EMAIL HOSTING
PHONE SERVICE
REDESIG

TRAVEL REIMBURSEMENT AND STIPEND
DATA ENTRY SERVICES
SOFTWARE AND SUPPORT
COMMUNICATIONS CONSULTING SERVICES
CAMPAIGN NEWSLETTER
E-MERCHANT FEES
PAYMENT FOR PAC FEDEX SHIPPING CHARGES
CONTRIBUTION TO A NONFEDERAL CAMPAIGN
MONTHLY SETUP CHARGE
MASS PAC ACCOUNTING/BOOKKEEPING EXP.
MASS PAC OFFICE SUPPORT EXPENSES
PAYROLL EXPENSES
MERCHANT SERVICE CHARGE
2020 PRIMARY
SERVICE CHARGE FEE
AUTO FUEL
GIFTS
SHIPPING CHARGES
TECH SUPPORT
CREDIT CARD: SEE BELOW
EXPENSES (SEE BELOW IF ITEMIZED)
STORAGE UNIT
BANK CHARGES
REIMBURSE FOR PAC ADMIN WORK-11/2018 AND 12/2018
CELLPHONE SERVICE
PAYMENT FOR PHONE LINES
PHOTOCOPIER SERVICES
MERCHANT PROCESSING FEE
MERCHANT SERVICE BANKCARD FEE
ALLOCATION REFUND OVER TRANSFER 11-14-18
ALLOCATION REFUND OVER TRANSFER 11-05-18
JANUARY SALARY
OFFICE ASSISTANCE LABOR
SOFTWARE SERVICES
PHONE.
STORAGE.
TICKETS TO EVENT
TRAVEL COSTS
EXPRESS MAIL
GAS REIMBURSEMENT FOR STAFF
FOOD FOR STAFF/ VOLUNTEERS
UTILITIES PAYMENT
FUNDRAISING CONSULTING FEE
MERCHANT BANK FEE
REFU

MERCHANT SERVICES FEES
IT CONSULTING SERVICES
PAC CC TRANSACTION FEES
SOFTWARE & SUPPORT
PAID POLITICAL CONTRIBUTION
INKIND FOR FUNDRAISING EXPENSE
VENDOR REFUND
CONTRACT SERVICES
POSTAGE AND SHIPPING
PRINTING AND SHIPPING
CREDIT CARD PAYMENT - SEE MEMO ENTRIES
BANK SERVICE AND CREDIT CARD FEES
VOIDED STALE/CHECK DATED 12/28/17
VOIDED STALE/CHECK DATED 12/1/17
VOIDED STALE/CHECK DATED 3/28/18
VOIDED STALE/CHECK DATED 8/22/16
VOIDED STALE/CHECK DATED 9/26/16
VOIDED STALE/CHECK DATED 1/18/17
VOIDED STALE/CHECK DATED 12/15/15
VOIDED STALE DATED CHECK DATED 7/13/17
VOIDED STALE/CHECK DATED 5/18/16
VOIDED STALE/CHECK DATED 2/9/16
VOIDED STALE/CHECK DATED 8/3/17
MONETARY CONTRIBUTION
COMPUTER (EQUIP/PROGRAMMING/SUPPORT)
TRAVEL AND ACCOMMODATIONS- REIMBURSEMENT
CELL PHONES
GRASSROOTS POLITICAL ORGANIZING SERVICES
CREDIT CARD PAYMENT (SEE BELOW)
EQUIPEMNT PURCHASE (SEE CAPITAL ONE)
BUSES FOR WOMEN'S MARCH
MONTHLY CONTRIBUTION
NON MONETARY DONATION TO NEVADA ADVOCATES FOR PLANNED PARENTHOOD AFF

ELECTRICITY
FACEBOOK ADS
SECURITY MONITORING SERVICE
EVENT REGISTRATION FEE
TREASURER
RESEARCH
MEMBERSHIP FEE
CREDIT CARD (SEE BELOW IF ITEMIZED)
TRAVEL - FOOD
MAIL CONSULTANT
PRINTING & POSTAGE
CAGING & ESCROW/SHIPPING EXP
DATA: DONOR DATA REPORT/LIST FULFILL. & MAINT/MAIL LIST PROCESSING
MAILING LIST RENTAL
MAILSHOP SERVICES/PRINTING EXP
CREATIVE, PROGRAM MGT FEE
MAILSHOP SVCS/POSTAGE & DELIVERY/SHIPPING EXPENSE
CONSULTING WORK
RELOCATION EXPENSES
DC GOVT HEALTH
SALARIES
MEDIA STRATEGY SERVICES
DIRECT MAIL LIST PROCESSING
LEADERSHIP BONUS
VOID - CONTRIBUTION SENT TO PAC & PREVIOUSLY REPORTED - CHECK NEVER CASHED
ADVOCACY CONSULTING
NEW JERSEY STATE ASSEMBLY
STAFF HOURS
PAYMENT:  SEE BELOW
ADVERTISING EXPENSE.
REFUND OF PRIMARY 2020 CONTRIBUTION.  ACTUAL DATE OF DISBURSEMENT IS 12/20/18.
SOFTWARE.  ACTUAL DATE OF DISBURSEMENT IS 12/17/18.
SOFTWARE. ACTUAL DATE OF DISBURSEMENT IS 11/26/18.
SOFTWARE.  ACTUAL DATE OF DISBURSEMENT IS 10/12/18.
TRAVEL EXPENSE - AIRFARE
TRAVEL EXPENSE - LOD

...

DATA MANAGEMENT SERVICES
COPE DONATION
ACCOUNTING SERVICES CONSULTING
CAMPAIGN MEETING
RENT AND PARKING
MEDIA
MEDIA CONSULTING / WEB SERVICE
FLORAL EXPENSE
FOOD/BEVERAGE/OFFICE SUPPLIES
FUNDRAISING CONSULTING AND EXPENSE REIMBURSEMENT
FOOD/BEVERAGE AND MEMBERSHIP DUES
DUES AND MEETING SPACE
HOUSE CONFERENCE RETREAT
SHIPPING & POSTAGE
REIMBURSEMENT-DETAIL BELOW IF ITEMIZED
CONSULTING/POLITICAL STRATEGY
PHONESVC
RENT ATTRIBUTED TO FEDERAL (15%)
STIPEND
CONSTITUENT GIFTS
HOTEL ACCOMODATIONS
CREDIT CARD PAYMENT - SEE MEMO BELOW
CAMPAIGN EVENT - MEALS
CONTRIBUTION REFUND: ORIGINAL CONTRIBUTION 12/27/18
CONTRIBUTION REFUND: ORIGINAL CONTRIBUTION 12/26/18
CONTRIBUTION REFUND - ORIGINAL CONTRIBUTION 12/27/2018
TRAVEL / FACILITY RENTAL / CATERING / FOOD/BEVERAGE
EVENT VENUE RENTAL
CONSULTANT - SCHEDULING
EVENT JANITORIAL SERVICES
FACILITY USAGE AND ACCOMODATIONS, NON-CONTRIBUTION ACCOUNT
RECEPTION, MEALS, LODGING
OPERATIONAL & STRATEGIC CONSULTING
REIMBURSEMENT FOR ENVELOPES
REIMBURSEMENT FOR P

REFUND - LODGING
VOID DONATION FROM 11/19/2018, NOT RE-ISSUED
DIRECT CONTIRIBUTION
REIMBURSEMENT OF ANDPAC EXPENSES
REGISTRATION FEES
IT SUPPORT/MAINTENANCE
VEHICLE LEASE
DONOR MEMENTOS
CONSULTANT FEE (FUNDRAISING)
VOID OF 9/2018 CHECK
AUTOMOBILE MAINTENANCE
POSTAGE-ADMINISTRATIVE
UTILITY
COMMUNICATIONS TRAINING
VOID - NAPOLITANO FOR CONGRESS
VOID OF 02.16.18 POLITICAL CONTRIBUTION
VOID OF 03.05.18 POLITICAL CONTRIBUTION
REIMBURSE TRAVEL - BELOW IF ITEMIZED
COMPLIANCE ACCOUNTING
POSTAL BOX RENTAL
NON-CONTRIBUTION ACCOUNT: TRAVEL REIMBURSEMENT - SEE BELOW IF ITEMIZED
NON-CONTRIBUTION ACCOUNT: TRAVEL
NON-CONTRIBUTION ACCOUNT: MILEAGE REIMBURSEMENT
NON-CONTRIBUTION ACCOUNT: TRAVEL & CATERING REIMBURSEMENT - SEE BELOW IF ITEMIZED
NON-CONTRIBUTION ACCOUNT: TRAVEL & CATERING REIMBURSEMENT- SEE BELOW IF ITEMIZED
NON-CONTRIBUTION ACCOUNT: EVENT, CATERING, POSTAGE, TRAVEL REIMBURSEMENT - SEE BELOW IF ITEMIZED
NON-CONTRIBUTION ACCOUNT: TRAVEL & OFFICE SUPPLIES REIMBURSEMENT- SEE BELOW IF ITEMIZE

TRAVEL VENDOR DETAIL FOR DISBURSEMENT TO VIVIAN CHANG
TRAVEL VENDOR DETAIL FOR DISBURSEMENT TO CHRISTOPHER BIGELOW
HEADQUARTERS UTILITIES
CONTRIBUTION-OH12
CONSULTANT - MEDIA
CONSULTANT - STRATEGY
CONSTITUENT SUPPLIES
CUSTOM BANNER
MOVING EXPENSE
BANNER
OFFICE LEASE SECURITY DEPOSIT
MEALS - OFFICIALLY CONNECTED
DATABASE SERVICE
CONSTITUENT MEAL
SOCIAL MEDIA CONSUTLING
EMAIL TOOLS
TRAIN TICKETS
FIELDWORK CONSULTING
MANAGEMENT AND COMMUNICATIONS CONSULTING, NON-CONTRIBUTION ACCOUNT
TICKETS FOR EVENT
NEWSPAPER AD
PRINTING - HOLIDAY CARDS
AUDIO VISUAL FOR SPEECH
AMEX PMT [SB17.31652]: MEETING EXPENSE: MEALS
INSURANCE EXPENSE - CAMPAIGN VEHICLE
CONSULTING SERVICES EXPENSE - COMMUNICATIONS
TELECOMMUNICATIONS EXPENSE
FUNDRAISER CONSULTING
W2 SERVICES
BANK FEE: TRANSFIRST DISCOUNT FEE
POST OFFICE BOX FEE
PAC EVENT TICKETS
FEDERAL ACCOUNT 2019
CONSTITUENT APPRECIATION
PAST DUE INVOICE MISSED FROM CLEAN UP
OFFICE MOVING SERVICES
PUBLIC RELATIONS
CATERING FOR CAMPAIGN EVENT
LEASE PAYMENT
FLIGHT R

NON-CONTRIBUTION ACCOUNT-CREDIT CARD PAYMENT
REFUND OF MONETARY CONTRIBUTION RECEIVED
IT - WEBSITE SERVICES
AMEX PMT [SB21B.7488]: MOBILE PHONE EXPENSE
EVENT COSTS
FUNDRAISING CONSULTING FEES & REIMBURSEMENTS
OPERATIONAL AND ORGANIZATIONAL CONSULTING
CREDIT CARD PURCHASES: SEE MEMOS
CAMPAIGN EVENT - CATERING
EVENT FOOD & BEVERAGES
KAUFMAN REIMBURSEMENT [SB.7125 ]: OFFICE SUPPLIES
AMEX PMT [SB17.31652]: TRAVEL: FOOD
IN-KIND - PAID FOR VOLUNTEER EVENT DINNER
PAC TRANSPORTATION, FOOD/BEVERAGE, HOST GIFTS
MONTHLY FEE- EMAIL SERVICE
MEETINGS/POLITICAL EVENTS
PAC FOOD/BEVERAGE
TRAVEL/CAR FARE
GENERAL CONSULTING SERVICES
REIMBURSED-MILEAGE
JOINT FUNDRAISING TRANSFER
TRANSPORTATION TO/FROM MEETINGS
*IN-KIND: LODGING
CREDIT CARD PAYMENT - SEE MEMOED
BUTTONS/BUMPERSTICKERS/SIGNS/TSHIRTS
BURLINGTON TELECOM
IN-KIND - VIDEO: JUST: A LOOKING 4 A HOME
NET PAYROLL
RECEPTION CATERER
TRAINING & PLANNING CONFERENCE
DESIGN FEES
2018 FUTA TAX
CATERING FOR MEETINGS AND FUNDRAISING EVENTS
ALLOCATION TRANSFER

BUSINESS INTERNET
FEDERAL INCOME TAX ON INTEREST INCOME
SHIPPING CHARGES FOR T-SHIRTS
2024 PRIMARY
TRANSFER OF STATE DUES TO AFFILITATED STATE PAC
TREVEL
RETURNED CHECK
PAC TREASURY SERVICES
PROFESSIONAL FEES-ACCOUNTING
 IN KIND -ANNUAL PO BOX RENTAL #189
PAC EVENT EXPENSE
PAYROLL: SEE MEMOS
PAYROLL SERVICE CHARGE
AMEX PMT [SB17.54035]: DELIVERY SERVICES
AMEX PMT [SB17.54035]: MEETING EXPENSE: MEALS
SALES TAX PAYABLE
REIMBURSEMENT [SB21B.44404]: TRAVEL: GROUND TRANSPORTATION
EVENT TABLE
INTERNET & PHONE
VOIDED 9/6/18 DISBURSEMENT
WAGES FOR ONLINE MANAGEMENT
RECEPTION FOOD AND BEVERAGE
STRATEGIC CAMP CONSULTING
REPORTING FEE
SECURITY SVC
UTILITIES-GAS
PEST CONTROL ANNUAL EXPENSE
PREPARATION OF BUSINESS PERS. PROPERTY TAX RETURN
CATERING & FACILITY FEES
FRANK HOFFMAN, STATE HOUSE 15TH LA
12/18 ACCT ANALYSIS FEE
ESTIMATED TAX PAYMENT
2019 NON-FEDERAL PAC CONTRIBUTION
PAYROLL GARNISHMENT
OFFICIALLY CONNECTED-RECEPTION-CATERING
OFFICIALLY CONNECTED-MEETINGS/MEALS
OFFICIALLY CONNECTED-SUPPOR

In [20]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 16723 items in vocab_frame


In [21]:
vocab_frame.head(50)

,words
digit,digital
consulting/fundrais,consulting/fundraising
account,account
analysi,analysis
chase,chase
void,voided
origin,original
date,date
void,voided
origin,original


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['Categories']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 1.01 s, sys: 6.86 ms, total: 1.02 s
Wall time: 1.02 s
(5034, 1)


In [24]:
terms = tfidf_vectorizer.get_feature_names()

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [27]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 206 ms, sys: 5.64 ms, total: 212 ms
Wall time: 218 ms


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  """Entry point for launching an IPython kernel.
